# Evaluation of experiments - union - DEMO
fistly define the UUID of the experiment to use

In [ ]:
experimentId=""

-------------------------------------------------------

In [ ]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#project variables
demopath=os.getcwd()

## load dreams

In [ ]:
samplerate=200  #Should rethink this

In [ ]:
signals, annotations, signalsMetadata = loadDREAMSSpindlesDemo(demopath)

In [ ]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)
len(annotations)

In [ ]:
minDuration=0.3
maxDuration=5
annotations=annotations[annotations.duration>minDuration].reset_index(drop=True)
annotations=annotations[annotations.duration<maxDuration].reset_index(drop=True)
len(annotations)

In [ ]:
signalsMetadata.head()

## Load experiment results

In [ ]:
experimentModels, featureSelection = loadExperiment(experimentId,demopath+"/DREAMS")

In [ ]:
experimentModels.head()

In [ ]:
featureSelection

## Optimal hyperparameter estimation with UNION criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [ ]:
experimentModels=experimentModels[experimentModels.criteriumName=='union'].reset_index(drop=True)
experimentModels

In [ ]:
auxThres=pd.DataFrame({
    'hyperThres':[0.1,0.2,0.3,0.4,0.5,0.6]
})
auxDepth=pd.DataFrame({
    'hyperDepth':[10,20,30,40,50,60]
})
auxDuration=pd.DataFrame({
    'hyperDuration':[0.3]
})
auxClose=pd.DataFrame({
    'hyperClose':[0.1]
})

hyperParams=pd.merge(auxThres,auxDepth,how='cross')
hyperParams=pd.merge(hyperParams,auxDuration,how='cross')
hyperParams=pd.merge(hyperParams,auxClose,how='cross')

hyperParams

In [ ]:
hyperExperimentModels=[]
hyperMeanF1=[]
hyperStdF1=[]

for ind_hyper,row_hyper in hyperParams.iterrows():
    print(ind_hyper)
    thisExperimentModels=experimentModels.copy()
    hyperThres=row_hyper.hyperThres
    hyperDepth=int(row_hyper.hyperDepth)
    hyperDuration=row_hyper.hyperDuration
    hyperClose=row_hyper.hyperClose
    
    meanF1=[]
    meanPrecision=[]
    meanRecall=[]

    stdF1=[]
    stdPrecision=[]
    stdRecall=[]

    for ind,row in thisExperimentModels.iterrows():
        #load model
        model=loadBooster(row.modelId,experimentId,demopath+'/DREAMS')
        #initialise lists
        rawF1s=[]
        rawPrecisions=[]
        rawRecalls=[]

        f1s=[]
        precisions=[]
        recalls=[]

        #iterate validation subjects
        for valSubjectId in row.val:
            #Define annotations criterium
            usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
            #Load features and labels
            valFeatures=loadFeatureMatrix([valSubjectId],featureSelection,signalsMetadata,samplerate,demopath)
            valLabels=loadLabelsVector([valSubjectId],usedAnnotations,signalsMetadata,samplerate)
            #Predict
            valDMatrix=xgb.DMatrix(data=valFeatures)
            probabilities=model.predict(valDMatrix,iteration_range=(0,hyperDepth))
            raw=probabilities>=hyperThres
            #Processed labels
            processed=labelingProcess(raw,hyperClose,hyperDuration,samplerate)
            gtAnnotations=labelVectorToAnnotations(valLabels,samplerate)
            detections=labelVectorToAnnotations(processed,samplerate)
            #Metrics
            f,r,p=annotationPairToMetrics(gtAnnotations,detections)
            
            #Metric appends
            f1s.append(f)
            precisions.append(p)
            recalls.append(r)

        #statistics of the metrics over the subjects of the validation set
        meanF1.append(np.mean(f1s))
        meanPrecision.append(np.mean(precisions))
        meanRecall.append(np.mean(recalls))

        stdF1.append(np.std(f1s))
        stdPrecision.append(np.std(precisions))
        stdRecall.append(np.std(recalls))
        
    thisExperimentModels['meanF1']=meanF1
    thisExperimentModels['meanPrecision']=meanPrecision
    thisExperimentModels['meanRecall']=meanRecall
    
    thisExperimentModels['stdF1']=stdF1
    thisExperimentModels['stdPrecision']=stdPrecision
    thisExperimentModels['stdRecall']=stdRecall

    hyperExperimentModels.append(thisExperimentModels)
    hyperMeanF1.append(np.mean(thisExperimentModels['meanF1']))
    hyperStdF1.append(np.std(thisExperimentModels['meanF1']))
    

In [ ]:
hyperParams['meanMeanF1']=hyperMeanF1
hyperParams['stdMeanF1']=hyperStdF1

In [ ]:
optimalInd=hyperParams[hyperParams.meanMeanF1==np.max(hyperParams.meanMeanF1)].index[0]
print("maximal mean score at:")
optimal=hyperParams.iloc[optimalInd]
optimal

## Graphical representation of the optimal point

In [ ]:
aux=hyperParams[(hyperParams.hyperDepth==optimal.hyperDepth)&(hyperParams.hyperDuration==optimal.hyperDuration)&
                (hyperParams.hyperClose==optimal.hyperClose)].reset_index(drop=True)
px.scatter(aux,x='hyperThres',y='meanMeanF1',error_y='stdMeanF1')

In [ ]:
aux=hyperParams[(hyperParams.hyperThres==optimal.hyperThres)&(hyperParams.hyperDuration==optimal.hyperDuration)&
                (hyperParams.hyperClose==optimal.hyperClose)].reset_index(drop=True)
px.scatter(aux,x='hyperDepth',y='meanMeanF1',error_y='stdMeanF1')